# Extend The TAO API to Support Datasets

- Events
    1. Support creation of datasets
    2. Support the `nextimage`, `cacheimage`, `notify` actions

## Setup imports

In [ ]:
import requests
import json

## Set up login

In [ ]:
monai_service = "<monai service API address>"
host_url = f"{monai_service}"
ngc_api_key = "<ngc keys>"

Object storage dataset basic info

In [ ]:
# Object storage server
manifest_url = "<manifest url>"
access_id = "<user name for the object storage>"
access_secret = "<secret for the object storage>"

Dicom dataset basic info

In [ ]:
# Dicom Server
dicom_web_endpoint = "<dicom web url>" # For example "http://127.0.0.1:8042/dicom-web".
dicom_client_id = ""    # If Authentication is enabled, then provide username
dicom_client_secret = "" # If Authentication is enabled, then provide password

## Login with NGC Key

In [ ]:
# Exchange NGC_API_KEY for JWT
data = json.dumps({"ngc_api_key": ngc_api_key})
response = requests.post(f"{host_url}/api/v1/login", data=data)#, verify=f"/usr/share/ca-certificates/nvidia/nvidia.crt")
print(response.status_code)
assert response.status_code in (200, 201)
assert "user_id" in response.json().keys()
user_id = response.json()["user_id"]
print("User ID",user_id)
assert "token" in response.json().keys()
token = response.json()["token"]
print("JWT",token)

# Set base URL
base_url = f"{host_url}/api/v1/users/{user_id}"
print("API Calls will be forwarded to",base_url)

headers = {"Authorization": f"Bearer {token}"}


## Create a object storage dataset

Based on object storage

In [ ]:
data = {
    "name": "MONAI_CLOUD",
    "description":"Object storage dataset",
    "type": "semantic_segmentation",
    "format": "monai",
    "client_url": manifest_url,
    "client_id": access_id,
    "client_secret": access_secret,
}
data=json.dumps(data)

endpoint = f"{base_url}/datasets"
print(endpoint)
print(headers)
response = requests.post(endpoint, data=data, headers=headers)
print(response.json())

if response.status_code == 201:
    res = response.json()
    dataset_id = res["id"]
    print("Dataset creation succeeded with dataset ID： ", dataset_id)
    print("---------------------------------\n")
    print(json.dumps(res, indent=2))
else:
    print(response.json())
    print(response)

Based on dicom

In [ ]:
data = {
    "name": "mydataset",
    "description":"a demo dataset",
    "type": "semantic_segmentation",
    "format": "monai",
    "client_url": f"{dicom_web_endpoint}",
    "client_id": f"{dicom_client_id}",
    "client_secret": f"{dicom_client_secret}",
}

endpoint = f"{base_url}/datasets"
response = requests.post(endpoint, json=data, headers=headers)
if response.status_code == 201:
    res = response.json()
    dataset_id = res["id"]
    print("Dataset creation succeeded with dataset ID: ", dataset_id)
    print("---------------------------------\n")
    print(json.dumps(res, indent=2))
else:
    print(response.json())
    print(response)

### Get the next image from the dataset. 

In [ ]:
data = {"action": "nextimage", "specs": {}}
endpoint = f"{base_url}/datasets/{dataset_id}/jobs"
response = requests.post(endpoint, json=data, headers=headers)

if response.status_code == 201:
    res = response.json()
    image_id = res["image"]
    print(f"Recommended Image to annotate: {image_id}")
    print("---------------------------------\n")
    print(json.dumps(res, indent=2))
else:
    print(response.json())
    print(response)

### Cache image

In [ ]:
specs = {
    "image": image_id,
    "ttl": 60,
}
data = {"action": "cacheimage", "specs": specs}
endpoint = f"{base_url}/datasets/{dataset_id}/jobs"
response = requests.post(endpoint, json=data, headers=headers)

if response.status_code == 201:
    res = response.json()
    print(json.dumps(res, indent=2))
else:
    print(response.json())
    print(response)

### Notify dataset

In [ ]:
# After uploading a Seg into cloud server
endpoint = f"{base_url}/datasets/{dataset_id}/jobs"
label_id = "12345"  # Fix This: Get This from cloud server for saved label
specs = {
    "added": {
        "image": image_id,
        "label": label_id,
    },
    "updated": [],
    "removed": [],
}

data = {"action": "notify", "specs": specs}

response = requests.post(endpoint, json=data, headers=headers)
if response.status_code == 201:
    print("Notified.")
else:
    print(response.json())
    print(response)

### Delete Datasets

In [ ]:
endpoint = f"{base_url}/datasets/{dataset_id}"
response = requests.delete(endpoint, headers=headers)
print(response)